Search
====

One of the most common (and most useful) operations on graphs is search. We said earlier that graphs are so powerful because they can naturally depict a lot of objects, and search made that even clearer. However, in order to do search, we translate the ***state*** **graph** to a ***search*** **tree**. This is a very important distinction!

Here is the state graph and adjacency matrix we will use:
<img src=state_graph.png width=450px>

The ***state graph*** is a representation of the problem, or the world in which the problem occurs. The ***search tree*** is a snapshot of our progress towards the goal.

One of the breakthroughs in artificial intelligence was the realization that we can use search in order to find the best next steps in a game. When Deep Blue beat Gary Kasparov in chess, much of its success was based on clever search implementations (see also http://fivethirtyeight.com/features/rage-against-the-machines/ for an interesting analysis).

A game can be modelled as a succession of states, and a goal is any state in which we win.
In order to find the best strategy, we depict each game state as a tree node, with the current state as root, and try to find the best path to a goal node. 

One of the important decisions we need to make is how to represent each state, i.e., what kind of information it contains (position of the pieces, whose turn it is, etc.). 
The children of each node are the states that correspond to the result of some action. So if we are at a point in the game where we have the choice between taking a magic sword or not, the child nodes of the current state are the one where we have the sword, and the one where we don't.

We traverse a search-tree by going down the tree to find the best outcome node (with whatever "best" is in our case).
We can then either return the best node we found, or the path (i.e., the other nodes and potentially the actions that were involved in reaching them) that were visited along the way.

The easiest way to search would be of course to simply look at each node in the tree until we have found our goal. While this is guaranteed to find the goal, it is impossible for most real-life applications. It would either take waaaay too long, or it would simply never finish: some search trees can be gigantic, or infinite (the number of states for a chess game is allegedly something like $10^{10^{50}}$).
One way around this is to limit the depth to which you search, and to keep track of the nodes you already visited.

In general, tree-search works as follows

<pre>
def search(tree, goal, strategy):
    <b>visited</b> = set()
    get start node
    add start node to <b>strategy</b> and <b>visited</b>

    while <b>strategy</b> is not empty:

        <b>current</b> = choose next node from <b>strategy</b>

        if <b>current</b> == <b>goal</b>:
            return <b>current</b>
        else:
            for child in <b>current</b>.children:
            if child is not in <b>visited</b>:
                <b>visited</b>.add(<b>current</b>)
                expand <b>strategy</b> with child

    return None

</pre>

This looks almost like Python, but not quite. The reason is that there are different search algorithms, and they mainly differ in the strategy they use, i.e., what kind of data structure they use in order to decide on the next node to visit (this is a popular job-interview question). We will look at some of the most well-known search algorithms. They apply to different problems, but they all use this basic structure, and only differ in the type of data structure they use for choosing the next state (this is a popular programming interview question).

We can encode the adjacency matrix in a `numpy` matrix.

In [2]:
import numpy as np
import pandas as pd

A = np.zeros((12,12), dtype=int)
edges = {1:[0], 2:[0], 3:[1,2,4], 4:[8,10], 5:[6], 7:[3,4,9], 8:[5], 9:[11], 10:[9,11]}
for start_state, goal_list in edges.items():
    for goal in goal_list:
        A[start_state, goal] = 1

A = pd.DataFrame(A)
A

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0
3,0,1,1,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,1,0
5,0,0,0,0,0,0,1,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,1,1,0,0,0,0,1,0,0
8,0,0,0,0,0,1,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,1


Constructing a Search Tree
----

In order to construct a search tree from a problem graph, we first need to choose a ***start node***. We can then use a fairly simple function that generates child (or ***successor***) nodes. In the simplest case, we can just use the problem graph's adjacency matrix.

Since the structure of the search tree depends on where we start, it will look different every time. For now, let's see how we search our way starting from node number 7. In that case, the search tree looks like this:
<img src=search_tree.png width=450px>

Depending on the size of the search space, we can compute the search tree before we start searching, by applying the child function ***recursively*** (i.e., we apply the same function to the children's children, and their children, and so forth), or use the successor function in the search algorithm to generate the child nodes online.

The successor function is the most important part of a node, and often, it's all you need. In general, though, search tree nodes need to have 5 components:

1. a representation of the state we are in at the node
2. the parent node (the one which generated this one)
3. the action that was used to get from the parent to this node
4. the cost of reaching this node from the start node
5. the number of steps taken to get to that node

Note that lot of these depend on the problem you are encoding. For a very principled and detailed discussion of search nodes, see Russel and Norvig, chapter 3.



Exercises
====

In [3]:
def get_children(graph, node):
    return np.nonzero(graph[node])[0].tolist()


In these exercises, you will implement two of the most common search algorithms. To jog your memory, and to enable you to check whether you get the correct result, here is the search tree we will use (rooted in 7).

<img src=search_tree.png width=450px>



1. Depth-First Search
---

The strategy in Depth-First Search (DFS) is to make a deep-dive: pick the first child node, look at its first child node, and so on. The way this is implemented is by a data structure called a ***stack***: whatever is put on last is the next thing we have to look at. It is also called a LIFO (Last In First Out). 

In Python, there is no dedicated stack structure. Instead, we use a list. We append at the end of te list, and we also get the next item by looking at the last item of the list. Rather than assigning the last item to a variable and then deleting the last list element, we use a function, `pop()`, that removes the last element from the list and returns it in one go.

Exercise 1
----

Implement DFS in the box below. You will want to use the `get_children` function defined above. Print out the stack at each iteration before you pop the next item. For starters, simply output the goal node once it has been found.


In [4]:
def dfs(graph, goal, start):
    # Your code here
    return None
                    
print(dfs(A, 5, 7))

None


Exercise 1.1
----

Modify you code to return the ***path*** to the goal node (i.e., a list of nodes) rather than just the goal node. You will have to change what goes on the stack and keep track of the current path.



In [12]:
def dfs_path(graph, goal, start):
    # Your code here
    return None
                    
print(dfs_path(A, 5, 7))

 None


2. Breadth-First Search
---

DFS is reasonably fast in practice, and more efficient if you have to search the entire tree anyway (why?), but it has a problem: if one of the branches of the tree is infinite, and therefore does not contain the solution, we will never get a solution - the search just goes on forever. 

Breadth-First Search (BFS) avoids this by going "level-by-level". It needs more memory, but it will find the solution.

Instead of a stack, we now use a ***queue***: whatever is put in first comes out first (FIFO: First In First Out). The only change we have to make in our code is to pop the first, rather than the last element of our list. Everything else stays the same.

Exercise 2
----

Implement BFS in the box below. Print out the queue at each iteration before you pop the next item. Return the path to the goal.


In [13]:
def bfs_path(graph, goal, start):
    # Your code here
    return None
                    
print("DFS", dfs_path(A, 5, 7))
print()
print("BFS", bfs_path(A, 5, 7))

DFS None

BFS None


Exercise 2.1
----

Rather than just the first best path, modify your code to return ***all*** paths that contain a solution. Try it with `bfs_path(A, 0, 7)`.

In [5]:
# Your code here

A\* Search
---

The two search algorithms you will implement in the exercises simply rely on the structure of the search tree, without including any information from the edge weights. Those algorithms are thus called *uninformed*, and they work on unweighted graphs.

In contrast, ***A\**** is an *informed* search algorithm, which uses a heuristic to estimate which states should be explored further. 

For each state $n$, we compute two measures, called $h(n)$ (the cost of getting from $n$ to the goal, or ***h***ome), and $g(n)$ (the cost of ***g***etting to $n$). Those two measured are added up to another measure, $f(n)$. Then, we sort all future states by their $f$-values, and expand the one with the lowest $f(n)$.

The core idea is that there is no sense in exploring bad states. The question is of course: how do we measure *bad*? This is up to the heuristics. The heuristics can be anything we want, as long as they don't give too pessimistic an estimate. That is, $f(n)$ should never be greater than the true cost.

In Python, we can use a list of tuples $(f(n), n)$ to find the best state.

A\* is used in a number of different applications, such as

* routing problems
* resource planning problems
* robot motion planning
* language analysis
* machine translation
* speech recognition
* games

See <a href="http://www.redblobgames.com/pathfinding/a-star/introduction.html">here</a> for a neat visualization of A\* search.